# ETL

O processo de ETL (Extract, Transform, Load) é uma parte crucial na análise do nosso Dataset. Ele nos permite extrair dados brutos, transformá-los em um formato adequado para análise e carregá-los em um sistema de armazenamento eficiente.

Nesse sentido, para implementar o processo de ETL, utilizamos um notebook Jupyter para facilitar a manipulação e visualização dos dados. O notebook está estruturado em três etapas principais:

1. **Extração (Extract)**: Nesta etapa, utilizaremos os dados brutos da camada raw. Utilizamos bibliotecas como `pandas` para ler e carregar os dados brutos.

2. **Transformação (Transform)**: Após a extração, os dados passam por um processo de limpeza e transformação. Isso inclui a remoção de duplicatas, preenchimento de valores ausentes e a aplicação de transformações de tipo, como conversão de tipos de dados e normalização.

3. **Carga (Load)**: Por fim, após o tratamento dos dados, o script vai criar um DDL para criar a tabela na camada silver e carregar os dados transformados nela.

## 1. Extração (Extract)

Nesta primeira etapa, realizamos a extração dos dados brutos da camada raw.
Utilizamos a biblioteca Pandas para ler o arquivo CSV com os dados brutos e carregá-lo em um DataFrame. 

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from pathlib import Path

raw_path = Path('../data/raw/currencies_data.csv')
silver_path = Path('../data/silver/silver_currencies_data.csv')

É possível visualizar abaixo os tipos de dados presentes no DataFrame após a extração:

In [2]:
df_raw = pd.read_csv(raw_path)
df_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 446176 entries, 0 to 446175
Data columns (total 24 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   cmcRank                   446176 non-null  int64  
 1   name                      446176 non-null  object 
 2   symbol                    446176 non-null  object 
 3   marketPairCount           446176 non-null  int64  
 4   circulatingSupply         446176 non-null  float64
 5   totalSupply               446176 non-null  float64
 6   maxSupply                 347894 non-null  float64
 7   isActive                  446176 non-null  int64  
 8   lastUpdated               446176 non-null  object 
 9   dateAdded                 446176 non-null  object 
 10  name.1                    446176 non-null  object 
 11  price                     446176 non-null  float64
 12  volume24h                 446176 non-null  float64
 13  marketCap                 446176 non-null  f

## 2. Transformação (Transform)

Após a extração dos dados, passamos para a etapa de transformação. Nesta fase, realizamos diversas operações para limpar e preparar os dados para análise.

Primeiro, são removidas duplicatas e colunas redundantes, garantindo a integridade das informações. Em seguida, convertemos os tipos de dados para formatos apropriados, tratamos valores ausentes e ajustamos colunas de data para o tipo datetime.

Além disso, aplicamos a função to_snake_case() para padronizar os nomes das colunas no formato snake_case, assegurando consistência e facilitando a manipulação futura dos dados.

In [3]:
import re

def to_snake_case(name: str) -> str:
    name = re.sub(r'(.)([A-Z][a-z]+)', r'\1_\2', name)
    name = re.sub(r'([a-z0-9])([A-Z])', r'\1_\2', name)
    name = re.sub(r'([a-zA-Z])(\d)', r'\1_\2', name)

    return name.lower()


df = df_raw.copy()
df = df.drop_duplicates()
df = df.drop(columns=["name.1"])

df["lastUpdated"] = pd.to_datetime(df["lastUpdated"], errors='coerce')
df["dateAdded"] = pd.to_datetime(df["dateAdded"], errors='coerce')
df["isActive"] = df["isActive"].astype(bool)
df["name"] = df["name"].astype("string")
df["symbol"] = df["symbol"].astype("string")

df.fillna(0, inplace=True)
df.columns = [to_snake_case(col) for col in df.columns]

df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 20225 entries, 0 to 427792
Data columns (total 23 columns):
 #   Column                       Non-Null Count  Dtype              
---  ------                       --------------  -----              
 0   cmc_rank                     20225 non-null  int64              
 1   name                         20225 non-null  string             
 2   symbol                       20225 non-null  string             
 3   market_pair_count            20225 non-null  int64              
 4   circulating_supply           20225 non-null  float64            
 5   total_supply                 20225 non-null  float64            
 6   max_supply                   20225 non-null  float64            
 7   is_active                    20225 non-null  bool               
 8   last_updated                 20225 non-null  datetime64[ns, UTC]
 9   date_added                   20225 non-null  datetime64[ns, UTC]
 10  price                        20225 non-null  float

In [4]:
silver_cryptocurrencies = Path('../data/silver/silver_currencies_data.csv')
df.to_csv(silver_cryptocurrencies, index=False)

## 3. Carga (Load)

Após o tratamento dos dados, utilizamos as colunas do DataFrame para criar um DDL que define a estrutura da tabela na camada silver.

In [5]:
def create_ddl_table(df: pd.DataFrame, table_name: str) -> str:
    dtype_mapping = {
        'int64': 'BIGINT',
        'float64': 'FLOAT',
        'bool': 'BOOLEAN',
        'datetime64[ns]': 'TIMESTAMP',
        'string': 'VARCHAR(255)',
        'object': 'TEXT'
    }

    ddl = f"CREATE TABLE IF NOT EXISTS {table_name} (\n"
    columns = []
    
    for col, dtype in df.dtypes.items():
        sql_type = dtype_mapping.get(str(dtype), 'TEXT')
        columns.append(f"    {col} {sql_type}")
    
    ddl += ",\n".join(columns)
    ddl += "\n);"
    
    return ddl

ddl_path = Path('../data/silver/ddl.sql')

with open(ddl_path, 'w') as f:
    f.write(create_ddl_table(df, 'currencies_data'))

Com a estrutura do DDL definida, o próximo passo é executar esse comando no banco de dados para criar a tabela. Em seguida, os dados transformados são carregados na tabela recém-criada, garantindo que estejam prontos para análise e consulta.

In [6]:
from dotenv import load_dotenv
import psycopg2
import os

load_dotenv('../.env')

def get_connection():
    try:
        conn = psycopg2.connect(
            host=os.getenv('DB_HOST','localhost'),
            database=os.getenv('POSTGRES_DB','postgres'),
            user=os.getenv('POSTGRES_USER','postgres'),
            password=os.getenv('POSTGRES_PASSWORD','postgres'),
            port=os.getenv('DB_PORT', 5432)
        )
        return conn
    except psycopg2.Error as e:
        print(f"Failed to connect to the database: {e}")
        raise e

In [7]:
with open(ddl_path, 'r') as f:
    ddl_sql = f.read()

conn = get_connection()
cursor = conn.cursor()

cursor.execute(ddl_sql)
conn.commit()

cursor.close()
conn.close()


print("Table created successfully.")


Table created successfully.


Agora basta iterar pelas linhas do DataFrame e inserir os dados na tabela criada.

In [8]:
from psycopg2 import sql

def insert_data(df: pd.DataFrame, table_name: str):
    conn = get_connection()
    cursor = conn.cursor()
    
    query = sql.SQL("INSERT INTO {table} ({fields}) VALUES ({placeholders})").format(
        table=sql.Identifier(table_name),
        fields=sql.SQL(', ').join(map(sql.Identifier, df.columns)),
        placeholders=sql.SQL(', ').join(sql.Placeholder() * len(df.columns))
    )
    
    for row in df.itertuples(index=False, name=None):
        cursor.execute(query, row)
    
    conn.commit()
    cursor.close()
    conn.close()
    print(f"Data inserted into {table_name} successfully.")

insert_data(df, 'currencies_data')


Data inserted into currencies_data successfully.


In [9]:
# VERSÃO COMPLETA COM VALIDAÇÕES

df = df_raw.copy()

print("=== TRATAMENTO DE DUPLICATAS RAW->SILVER ===")
print(f"Registros brutos: {len(df)}")

# 1. Garantir que as colunas-chave estão no formato correto
df['name'] = df['name'].astype(str)
df['lastUpdated'] = pd.to_datetime(df['lastUpdated'], errors='coerce')

# 2. Análise detalhada das duplicatas
print("\n--- ANÁLISE DE DUPLICATAS ---")
key_columns = ['name', 'lastUpdated']
duplicate_mask = df.duplicated(subset=key_columns, keep=False)
duplicate_groups = df[duplicate_mask].groupby(key_columns).size()

if len(duplicate_groups) > 0:
    print(f"Grupos duplicados encontrados: {len(duplicate_groups)}")
    print(f"Total registros duplicados: {duplicate_mask.sum()}")
    print(f"Exemplo de grupos duplicados:")
    print(duplicate_groups.head(10))
    
    # Verificar se há diferenças nos dados duplicados
    sample_dup = df[duplicate_mask].sort_values(key_columns).head(10)
    print(f"\nExemplo de registros duplicados:")
    print(sample_dup[['name', 'lastUpdated', 'price', 'marketCap']])
else:
    print("Nenhuma duplicata encontrada.")

print(f"\n--- REMOVENDO DUPLICATAS ---")
df_clean = df.sort_values('marketCap', ascending=False).drop_duplicates(subset=key_columns, keep='first')

print(f"Registros após limpeza: {len(df_clean)}")
print(f"Duplicatas removidas: {len(df) - len(df_clean)}")

# 4. Validação final
remaining_duplicates = df_clean.duplicated(subset=key_columns).sum()
print(f"Duplicatas restantes após limpeza: {remaining_duplicates}")

if remaining_duplicates == 0:
    print("Dados limpos - sem duplicatas!")
else:
    print("Ainda há duplicatas!")

df = df_clean

print(f"\n--- CONTINUANDO ETL COM {len(df)} REGISTROS ---")
df = df.drop(columns=["name.1"])


print("=== VALIDAÇÃO FINAL SILVER ===")
print(f"Total registros silver: {len(df)}")

# Verificar unicidade
unique_combinations = df[['name', 'lastUpdated']].drop_duplicates().shape[0]
print(f"Combinações únicas (name + lastUpdated): {unique_combinations}")

if len(df) == unique_combinations:
    print("Dados silver consistentes - sem duplicatas!")
else:
    print(f"PROBLEMA: {len(df) - unique_combinations} duplicatas ainda existem")

print(f"\n--- ESTATÍSTICAS SILVER ---")
print(f"Moedas únicas: {df['name'].nunique()}")
print(f"Timestamps únicos: {df['lastUpdated'].nunique()}")
print(f"Período: {df['lastUpdated'].min()} a {df['lastUpdated'].max()}")

=== TRATAMENTO DE DUPLICATAS RAW->SILVER ===
Registros brutos: 446176

--- ANÁLISE DE DUPLICATAS ---
Grupos duplicados encontrados: 19754
Total registros duplicados: 445862
Exemplo de grupos duplicados:
name             lastUpdated              
$BABY PEPE COIN  2023-09-04 15:00:00+00:00    23
$CROOGE          2023-09-04 15:00:00+00:00     5
$LAMBO           2023-09-04 15:00:00+00:00     7
$USDEBT          2023-09-04 15:00:00+00:00     6
$X               2023-09-04 14:59:00+00:00    18
                 2023-09-04 15:00:00+00:00    25
.Alpha           2023-09-04 15:00:00+00:00    12
00 Token         2023-09-04 14:59:00+00:00    31
                 2023-09-04 15:00:00+00:00    25
01coin           2023-09-04 15:00:00+00:00    20
dtype: int64

Exemplo de registros duplicados:
                   name               lastUpdated         price  marketCap
262701  $BABY PEPE COIN 2023-09-04 15:00:00+00:00  1.761092e-11        0.0
269900  $BABY PEPE COIN 2023-09-04 15:00:00+00:00  1.761092e-11    